<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
#copy log file
!cp '/content/drive/MyDrive/zHDFS2/logs_train' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_ntest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_atest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/log2index' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/reduced_embeddings' '/content/'


In [21]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
device

device(type='cuda')

In [23]:
with open('/content/reduced_embeddings', 'r') as json_file:
    embeddings = json.load(json_file)

In [24]:
len(embeddings[0])


10

In [25]:
# Create a tensor with shape [4, 1, 16]
original_tensor = torch.randn(4, 1, 16)

# Reshape it to [2, 1, 32] by combining the first two rows
reshaped_tensor = torch.cat([original_tensor[:2], original_tensor[2:]], dim=2).view(2, 1, 32)

# Check the shape of the reshaped tensor
print(reshaped_tensor.shape)  # Should print torch.Size([2, 1, 32])

torch.Size([2, 1, 32])


In [26]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, sequence_length, dropout_prob, device): #input_dim= embeddings_dim
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.device = device
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=False, dropout=dropout_prob)
        self.decoder = nn.LSTM(input_dim, hidden_dim , num_layers, batch_first=True, dropout=dropout_prob)
        # Define a linear layer to reduce dimensionality of output of cells
        self.projection_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        batch_size = x.shape[0]
        _, (hidden, cell) = self.encoder(x)


        #tensor to store decoder outputs
        outputs = torch.zeros(batch_size, self.sequence_length, self.input_dim).to(self.device)

        #first input to the decoder is the zero tokens
        input_decoder = torch.zeros(batch_size, 1, self.input_dim).to(self.device)

        for t in range(1,self.sequence_length):
          #insert input token embedding, previous hidden and previous cell states
          #receive output tensor (predictions) and new hidden and cell states
          output,(hidden, cell) = self.decoder(input_decoder, (hidden, cell))
          output = self.projection_layer(output)
          outputs[:, t] = output.squeeze(dim=1)
          # input for the next cell will be output of this cell
          input = output
        return outputs

In [27]:
sequence_length = 5
input_dim = 10 #embedding vector dimension
hidden_dim = 32
num_layers = 2
dropout_prob = 0.2

model = LSTMAutoencoder(input_dim, hidden_dim, 2, sequence_length, dropout_prob, device).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(10, 32, num_layers=2, batch_first=True, dropout=0.2)
  (decoder): LSTM(10, 32, num_layers=2, batch_first=True, dropout=0.2)
  (projection_layer): Linear(in_features=32, out_features=10, bias=True)
)

In [28]:
data = torch.randn(2,sequence_length, input_dim).to(device) #batch_size=1
print(data.shape)
print(model(data).shape)

torch.Size([2, 5, 10])
torch.Size([2, 5, 10])


In [29]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 28490


In [30]:

name = 'logs_train'
window_size = sequence_length
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 446578
Number of seqs(logs_train): 6936890


In [31]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
len(dataloader) # regarding to batch size

27098

In [32]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([256, 5, 10])


In [33]:
def adjust_learning_rate(optimizer, epoch, lr_step=(10,15,20,25), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [36]:
num_epochs = 30
learning_rate = 0.0001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [37]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_dim).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Epoch [1/30], train_loss: 0.0058
Epoch [2/30], train_loss: 0.0058


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Autoencoder3_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [38]:
len(embeddings)

52

In [39]:
embeddings.append([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [42]:
embeddings[len(embeddings)-1]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [41]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            line = line + [len(embeddings)-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by 30 a template with zeros vector
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [43]:
test_normal_loader, test_normal_length = generate('logs_ntest')
test_abnormal_loader, test_abnormal_length = generate('logs_atest')

Number of sessions(logs_ntest): 1091
Number of sessions(logs_atest): 4126


In [44]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in test_normal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_dim).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in test_abnormal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_dim).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [45]:
threshold = [0.00702,0.00703,0.00706,0.00707,0.00714]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.00702
elapsed_time: 99.574s
false positive (FP): 501, false negative (FN): 1984, Precision: 96.737%, Recall: 88.217%, F1-measure: 92.281%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.00703
elapsed_time: 97.954s
false positive (FP): 501, false negative (FN): 1984, Precision: 96.737%, Recall: 88.217%, F1-measure: 92.281%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.00706
elapsed_time: 98.182s
false positive (FP): 500, false negative (FN): 1984, Precision: 96.744%, Recall: 88.217%, F1-measure: 92.284%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.00707
elapsed_time: 97.135s
false positive (FP): 500, false negative (FN): 1984, Precision: 96.744%, Recall: 88.217%, F1-measure: 92.284%
Finished Predicting
------------

In [ ]:
threshold = [0.00726,0.00727,0.00728,0.00729]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)